In [ ]:
import os
os.chdir('..')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearnex import patch_sklearn
patch_sklearn()

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [ ]:
original_data = pd.read_csv('data/dataset.csv')

df = original_data.copy()
df.head()

In [ ]:
df.shape

### Handelling Missing Values

In [ ]:
# Filling with mean - 

missing_val_columns = ['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Water Temperature', 'Air Temperature']

for col in missing_val_columns:
    df[col].fillna(df[col].mean(), inplace = True)

In [ ]:
# Filling with mapping

df['Color'].fillna('Near Colorless', inplace=True)
color_mapping = df.groupby('Color')['Color'].transform('count') / len(df)

df['Color'] = color_mapping

In [ ]:
# Filling with mode

df['Source'] = df['Source'].fillna('Stream')

encoder = OrdinalEncoder()
df['Source'] = encoder.fit_transform(df[['Source']])

In [ ]:
# Deleting non important columns

del df['Day']
del df['Index']
del df['Month']
del df['Time of Day']

### Feature Engineering

In [ ]:
# Binning Iron based on the histogram (Low, Moderate and High)

bin_edges = [0, 0.1, 1, 20]
bin_labels = [0, 0.4, 1]
df['Iron_Bin'] = pd.cut(df['Iron'], bins = bin_edges, labels = bin_labels)

# Nitrate
bin_edges = [0, 1, 5, 100] 
df['Nitrate_Bin'] = pd.cut(df['Nitrate'], bins=bin_edges, labels=bin_labels)

# Copper
bin_edges = [0, 0.02, 1, 20]
df['Copper_Bin'] = pd.cut(df['Copper'], bins=bin_edges, labels=bin_labels)

### Validation Strategy

In [ ]:
# Choosing two validation datasets, in order to get a better idea of the model performance.
# Taking only 10% of the data for validation because of the very large size of the dataset.

X = df.drop('Target', axis = 1)
y = df['Target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 42, shuffle = True, stratify = y)
X_val_1, X_val_2, y_val_1, y_val_2 = train_test_split(X_val, y_val, test_size = 0.5, random_state = 42, stratify = y_val)

X_train.shape, X_val_1.shape, X_val_2.shape, y_train.shape, y_val_1.shape, y_val_2.shape

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_1_scaled = scaler.transform(X_val_1)
X_val_2_scaled = scaler.transform(X_val_2)

In [ ]:
X_train.info()

### Training

In [ ]:
import daal4py as d4p
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, ConfusionMatrixDisplay, f1_score

In [ ]:
clf = LGBMClassifier(objective = 'binary',
                     metric = 'binary_logloss',
                     random_state = 42,
                     n_jobs = -1,
                     force_row_wise = True)

callback = lgb.early_stopping(stopping_rounds = 10)
clf.fit(X_train_scaled, y_train, eval_set=[(X_val_2_scaled, y_val_2)], callbacks = [callback])

In [ ]:
print('predicting -')
y_pred_1 = clf.predict(X_val_1_scaled)
y_pred_2 = clf.predict(X_val_2_scaled)

print('Accuracy on validation set 1: ', accuracy_score(y_val_1, y_pred_1))
print('Accuracy on validation set 2: ', accuracy_score(y_val_2, y_pred_2))
print('Recall on validation set 1: ', recall_score(y_val_1, y_pred_1))
print('Recall on validation set 2: ', recall_score(y_val_2, y_pred_2))
print('F1 on validation set 1: ', f1_score(y_val_1, y_pred_1))
print('F1 on validation set 2: ', f1_score(y_val_2, y_pred_2))

In [ ]:
plt.bar(X_train.columns, clf.feature_importances_)
plt.xticks(rotation = 90)

plt.show()

In [ ]:
cm = confusion_matrix(y_val_1, y_pred_1)
ConfusionMatrixDisplay(cm).plot()

### Serialization

In [ ]:
import joblib

with open('model/model.joblib','wb') as out:
    joblib.dump(clf, out)

joblib.dump(scaler, 'model/scaler.joblib')
joblib.dump(encoder, 'model/ordinal_encoder.joblib')

In [ ]:
df.head()